In [33]:
import os
import json
def extract_funcdef_and_clis(json_path):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        func_def = data.get("FuncDef", "").strip()
        clis = data.get("CLIs", [])
        clis_text = "\n".join([cli for cli in clis])
        return func_def , clis_text, "CLIs:\n" + clis_text + "\nFuncDef:\n" + func_def
    except Exception as e:
        print(f"Error processing {json_path}: {e}")
        return None

def collect_all_texts(root_dir):
    """
    遍历目录下所有JSON文件，返回每个文件中FuncDef和CLIs拼接后的字符串列表
    """
    result_dict = {'cli':[],'def':[],'text':[]}

    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".json"):
                json_path = os.path.join(subdir, file)
                output = extract_funcdef_and_clis(json_path)
                if output:
                    result_dict['cli'].append(output[1])
                    result_dict['def'].append(output[0])
                    result_dict['text'].append(output[2])
    return result_dict

In [34]:
huawei_texts = collect_all_texts('./BGP配置命令行数据/huawei')
nokia_texts = collect_all_texts('./BGP配置命令行数据/nokia')


In [37]:
print(len(huawei_texts))
print(len(nokia_texts))

3
3


In [38]:
huawei_texts

{'cli': ['display bgp link-state unicast peer\ndisplay bgp link-state unicast slow-peer',
  'reflector cluster-id { <cluster-id-value> | <cluster-id-ipv4> }\nundo reflector cluster-id',
  'peer <group-name> enable\nundo peer <group-name> enable',
  'peer <ipv4-address> enable\nundo peer <ipv4-address> enable',
  'peer <ipv6-address> reflect-client\nundo peer <ipv6-address> reflect-client',
  'peer <ipv6-address> update-group-independent enable\npeer <ipv6-address> update-group-independent disable\nundo peer <ipv6-address> update-group-independent enable\nundo peer <ipv6-address> update-group-independent disable',
  'peer <ipv6-address> route-update-interval <interval>\nundo peer <ipv6-address> route-update-interval',
  'display bgp link-state unicast peer [ <ipv4-address> | <ipv6-address> ] verbose',
  'display bgp link-state unicast routing-table [ peer { <ipv4-address> | <ipv6-address> } { received-routes | advertised-routes } ] [ type { node | link | ipv4-prefix | ipv6-prefix | te-p

In [39]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# 1. 加载Embedding模型
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. 编码文本
emb_A = model.encode(huawei_texts['text'], convert_to_tensor=True)
emb_B = model.encode(nokia_texts['text'], convert_to_tensor=True)

# 3. 计算余弦相似度
similarity_matrix = cosine_similarity(emb_A.cpu(), emb_B.cpu())

top_k = 2  # 每个A中选2个最相似的B
for i, sim_row in enumerate(similarity_matrix):
    top_k_idx = np.argsort(sim_row)[::-1][:top_k]
    print('huawei命令')

    print(huawei_texts['cli'][i])
    for idx in top_k_idx:

        print('nokia命令')
        print(nokia_texts['cli'][idx])
        print(f"相似度：{sim_row[idx]:.4f}")

        print("-------")

    if i>10:
        break

In [45]:
top_k = 2  # 每个A中选2个最相似的B
for i, sim_row in enumerate(similarity_matrix):
    top_k_idx = np.argsort(sim_row)[::-1][:top_k]
    print('huawei命令')

    print(huawei_texts['cli'][i])
    for idx in top_k_idx:

        print('nokia命令')
        print(nokia_texts['cli'][idx])
        print(f"相似度：{sim_row[idx]:.4f}")

        print("-------")

    if i>10:
        break

huawei命令
display bgp link-state unicast peer
display bgp link-state unicast slow-peer
nokia命令
initial-send-delay-zero
no initial-send-delay-zero
相似度：0.7143
-------
nokia命令
type { internal | external }
no type { internal | external }
相似度：0.7117
-------
huawei命令
reflector cluster-id { <cluster-id-value> | <cluster-id-ipv4> }
undo reflector cluster-id
nokia命令
cluster <cluster-id> orr-location <location-id> [ allow-local-fallback ]
cluster <cluster-id>
no cluster
相似度：0.6619
-------
nokia命令
location <location-id> [ primary-ip-address <ipv4-address> ] [ secondary-ip-address <ipv4-address> ] [ tertiary-ip-address <ipv4-address> ]
相似度：0.5651
-------
huawei命令
peer <group-name> enable
undo peer <group-name> enable
nokia命令
group <name>
no group <name>
相似度：0.7587
-------
nokia命令
enable-peer-tracking
no enable-peer-tracking
相似度：0.7290
-------
huawei命令
peer <ipv4-address> enable
undo peer <ipv4-address> enable
nokia命令
enable-peer-tracking
no enable-peer-tracking
相似度：0.7403
-------
nokia命令
secondary-

In [16]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

def extract_commands_from_json(json_str):
    """
    从JSON字符串中提取commands列表。
    :param json_str: 纯JSON字符串
    :return: 命令字符串列表，若不存在则返回空列表
    """
    try:
        obj = json.loads(json_str)
        return obj.get('commands', [])
    except json.JSONDecodeError:
        return []

In [17]:
model_name = './qwen2.5'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16)
model.eval()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb):

In [4]:
model = PeftModel.from_pretrained(model,"./qwen",adapter_name='expert',torch_dtype=torch.float16)

In [21]:
def get_response(input_text, lora = True):
    # system
    system = '''
You are a router command assistant. Please generate the required router configuration commands based on my input, and output ONLY in JSON format as shown below:

{
  "commands": [
    "command 1",
    "command 2",
    "command 3"
    // The number of commands may vary depending on the requirements
  ]
}

Do not provide any explanation, description, or code block markers. Only return the pure JSON object.
'''
    messages = [{"role": "system","content":system},{"role": "user", "content": input_text}]
    # messages = [{"role": "user", "content": input_text}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(inputs, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            # adapter_names = ['__base__']
            adapter_names = ['expert'] if lora else ['__base__']
        )
    return tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)

In [23]:
# input_text = "In a Huawei router, how to configure the routing update interval to 50 seconds for a BGP peer with the IP address 192.168.1.4 in BGP LS unicast address family? My AS is 65001, and the peer AS is 123"
input_text = "On a Huawei router, which command is used to set the interval to 25 seconds for transmitting routing updates with the same prefix to a BGP peer at the IPv4 address 10.10.1.2? Provide me all commands"
# input_text = "如何在华为路由器上使用命令查看从地址为 192.0.2.2 的对端接收的 BGP-LS IPv4 路由？"

In [24]:
output_json = get_response(input_text,lora=True)
command_list = extract_commands_from_json(output_json)

In [46]:
command_list

['peer 10.10.1.2 route-update-interval 25']

In [50]:
class HuaweiCommandRetriever:
    def __init__(self, huawei_commands, model_name='all-MiniLM-L6-v2'):

        self.huawei_commands = huawei_commands
        self.model = SentenceTransformer(model_name)
        self.embeddings = self.model.encode([cmd for cmd in self.huawei_commands['cli']], convert_to_tensor=True)

    def retrieve(self, query, top_k=3):
        if isinstance(query, str):
            query_emb = self.model.encode([query], convert_to_tensor=True)
        else:
            raise ValueError("Query should be a string.")

        # 余弦相似度
        sim_scores = cosine_similarity(query_emb.cpu(), self.embeddings.cpu())[0]
        top_indices = np.argsort(sim_scores)[::-1][:top_k]

        results = []
        for idx in top_indices:
            results.append((float(sim_scores[idx]), self.huawei_commands['cli'][idx]))
        return results



retriever = HuaweiCommandRetriever(huawei_texts,'CyCraftAI/CmdCaliper-base')
llm_command = 'peer 10.10.1.2 route-update-interval 25'
top_matches = retriever.retrieve(llm_command, top_k=2)

for similarity, cmd in top_matches:
    print(f"华为命令: {cmd}（相似度: {similarity:.4f}）")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

You try to use a model that was created with version 3.1.1, however, your version is 2.6.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

华为命令: peer <group-name> route-update-interval <interval>
undo peer <group-name> route-update-interval（相似度: 0.5968）
华为命令: peer <group-name> route-update-interval <interval>
undo peer <group-name> route-update-interval（相似度: 0.5968）
